In [ ]:
from datetime import datetime
import logging
from pathlib import Path
import sys

from dateutil.parser import parse
from eodag import EODataAccessGateway, SearchResult
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon

sys.path.append(str(Path("..").resolve()))

from big_tide_prediction import tide_interpolation, tide_prediction
from coastline_change_functions import (
    coregistration,
    create_transects,
    db_scale,
    filter_tide,
    intersection_percent,
    lee_filter,
    rescale,
    segmentation,
    smooth_linestring,
    subpixel_contours,
    transect_analysis,
)


In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s: %(message)s")

logger = logging.getLogger(__name__)

In [ ]:
# parameters
REGION_ID = 679
START_DATE = "2015-01-01"
STOP_DATE = "2022-12-31" # datetime.today().date().isoformat()
TIDE_TYPE = "mean" # option: ht, lt, mean

In [ ]:
region_path = Path("../region/coastal_grids.geojson")
point_path = Path("../region/coastal_points.geojson")

region_gdf = gpd.read_file(region_path)
point_gdf = gpd.read_file(point_path)

In [ ]:
# filter_region = region_gdf.query("province == 'BALI'")
# centroid = filter_region.unary_union.centroid

# m = region_gdf.explore(
#     location=[centroid.y, centroid.x],
#     zoom_start=8,
#     style_kwds={"fillOpacity": 0, "color": "red", "linewidth": 10},
# )
# for i, row in region_gdf.iterrows():
#     centroid = row.geometry.centroid
#     folium.Marker(
#         location=[centroid.y, centroid.x],
#         icon=folium.DivIcon(
#             html=f'<div style="font-size: 12pt">{i+1}</div>'
#         )
#     ).add_to(m)

# folium.GeoJson(point_gdf.to_json()).add_to(m)

# m


In [ ]:
selected_region = region_gdf.loc[[REGION_ID-1]]
selected_point = point_gdf.loc[[REGION_ID-1]]

m = selected_region.explore(style_kwds={"fillOpacity": 0, "color": "red"})
folium.GeoJson(selected_point.to_json()).add_to(m)
m

In [ ]:
dag = EODataAccessGateway()

provider_path = Path(
    "/home/barata-serv/otomatisasi_barata/coastline/provider/sara_provider.yaml"
)

with open(provider_path) as f:
    dag.update_providers_config(f.read())

dag.set_preferred_provider("sara")
logger.info(f"Preferred provider: {dag.get_preferred_provider()}")

In [ ]:
selected_region_geom = selected_region.unary_union

search_params = {
    "productType": "S1_SAR_GRD",
    "start": START_DATE,
    "end": STOP_DATE,
    # "geom": {"lonmin": 114.573354951, "lonmax": 114.637952680, "latmin": -8.150957658, "latmax": -8.097021690},
    "geom": selected_region_geom,
}

search_results = (
    dag.search_all(**search_params)
    .filter_property(polarizationMode="VH,VV")
    .filter_overlap(selected_region_geom, minimum_overlap=98)
)


In [ ]:
def create_search_result_map(
    search_results: SearchResult, extent: Polygon
) -> folium.Map:
    """Small utility to create an interactive map with folium
    that displays an extent in red and EO Producs in blue"""
    result_gdf = gpd.GeoDataFrame.from_features(
        search_results.as_geojson_object(), crs="EPSG:4326"
    )

    fmap = result_gdf[
        [
            "platform",
            "orbitDirection",
            "sensorMode",
            "startTimeFromAscendingNode",
            "polarizationMode",
            "geometry"
        ]
    ].explore(
        location=[extent.centroid.y, extent.centroid.x],
        zoom_start=8,
        style_kwds=dict(color="blue", fillOpacity=0, linewidth=1),
    )

    # folium.GeoJson(
    #     search_results, style_function=lambda x: dict(fill=False, color="blue")
    # ).add_to(fmap)

    folium.GeoJson(extent, style_function=lambda x: dict(color="red")).add_to(fmap)
    
    return fmap


In [ ]:
create_search_result_map(search_results, selected_region_geom)

In [ ]:
time_list = sorted(
    [
        pd.to_datetime(
            result.properties["startTimeFromAscendingNode"], utc=True
        )
        for result in search_results
    ]
)

In [ ]:
x = selected_point.unary_union.centroid.x
y = selected_point.unary_union.centroid.y

In [ ]:
output_dir = Path("../output") / f"{REGION_ID:04d}"
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
tide_path = output_dir / f'{REGION_ID:04d}_tide.csv'

if not tide_path.exists():
    start_date_data = time_list[0]
    stop_date_data = time_list[-1]
    tide_df = tide_prediction(x, y, start_date_data, stop_date_data)
    interp_tide_df = tide_interpolation(tide_df, time_list)
    interp_tide_df.reset_index(inplace=True)
    interp_tide_df.rename(columns={"index": "datetime"}, inplace=True)
    interp_tide_df.to_csv(tide_path, index=False)
else:
    interp_tide_df = pd.read_csv(tide_path)

In [ ]:
tide_list = interp_tide_df['level'].tolist()

In [ ]:
lt = np.min(tide_list)
ht = np.max(tide_list)
mean = np.mean(tide_list)

logger.info(f"Low tide: {lt} | High tide: {ht} | Mean tide: {mean}")

In [ ]:
tide_results = []
for result, tide in zip(search_results, tide_list):
    result.properties["tide"] = tide
    tide_results.append(result)

tide_results = SearchResult(sorted(tide_results, key=lambda x: x.properties["startTimeFromAscendingNode"]))

In [ ]:
from itertools import groupby

grouped_results = [
    SearchResult(group)
    for _, group in groupby(
        tide_results,
        key=lambda x: parse(x.properties["startTimeFromAscendingNode"]).year,
    )
]

In [ ]:
lt_results = SearchResult(
    [
        sorted(group, key=lambda x: abs(x.properties["tide"] - lt))[0]
        for group in grouped_results
    ]
)
ht_results = SearchResult(
    [
        sorted(group, key=lambda x: abs(x.properties["tide"] - ht))[0]
        for group in grouped_results
    ]
)
mean_results = SearchResult(
    [
        sorted(group, key=lambda x: abs(x.properties["tide"] - mean))[0]
        for group in grouped_results
    ]
)


In [ ]:
tide_s1_results_dict = {
    "ht": ht_results,
    "lt": lt_results,
    "mean": mean_results
}

In [ ]:
def download_results(results: SearchResult, base_dir: Path) -> None:
    for result in results:
        platform = result.properties["platform"].lower()
        product_type = result.properties["productType"].lower()
        sensor_mode = result.properties["sensorMode"].lower()
        sensing_time = parse(result.properties["startTimeFromAscendingNode"])
        year = sensing_time.year
        month = sensing_time.month
        day = sensing_time.day

        output_dir = base_dir.joinpath(
            platform,
            f"{platform}_{sensor_mode}_{product_type}",
            str(year),
            f"{month:02d}",
            f"{day:02d}",
        )

        dag.download(result, outputs_prefix=output_dir, extract=False)


In [ ]:
tide_s1_results = tide_s1_results_dict[TIDE_TYPE]

result_path = Path("../output") / f"{REGION_ID:04d}" / f"{REGION_ID:04d}_s1_{TIDE_TYPE}.geojson"
result_path.parent.mkdir(parents=True, exist_ok=True)
dag.serialize(tide_s1_results, result_path)

In [ ]:
BASE_DIR = Path("/home/barata-serv/otomatisasi_barata/datasets/")

download_results(tide_s1_results, BASE_DIR)